## Import

In [18]:
import os

import csv
%matplotlib inline
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import eli5
import shap
from xgboost import XGBClassifier
from sklearn.feature_selection import RFE, RFECV
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score,
    ConfusionMatrixDisplay,
    classification_report,
    confusion_matrix,
    f1_score,
    make_scorer,
    plot_confusion_matrix,
    recall_score,
    mean_absolute_error
)
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.svm import SVC

In [19]:
df = pd.read_csv("Players_17_18.csv", index_col="playerId")

## Explaratory Data Analysis

In [20]:
# Import the data dictionary which describes the data
with open("MoneyPuckDataDictionaryForPlayers.csv", newline='') as csvfile:
    r = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in r:
        print(''.join(row))

*****MoneyPuck.com Player and Team Data******

Please reach out through MoneyPuck.com if you have any feedback
No guarantees are made to the quality of the data. NHL data is known to have issues and biases.
You are welcome to use this data in any work. Just please cite MoneyPuck.com

Below are a description of general terms used in the data as well as a data dictionary below it:



General TermsDescription
Expected Goals"The sum of the probabilities of unblocked shot attempts being goals. For example a rebound shot in the slot may be worth 0.5 expected goals while a shot from the blueline while short handed may be worth 0.01 expected goals. The expected value of each shot attempt is calculated by the MoneyPuck Expected Goals model. Expected goals is commonly abbreviated as ""xGoals"". Blocked shot attempts are valued at 0 xGoals. See more here: http://moneypuck.com/about.htm#shotModel"
Score AdjustedAdjusts metrics to gives more credit to away teams and teams with large leads.
Flurry A

In [21]:
pd.set_option('display.max_colwidth', None, "display.max_rows", None)
explanations = pd.read_csv("MoneyPuckDataDictionaryForPlayers.csv").drop(labels=range(25),axis=0)
renamed = {"*****MoneyPuck.com Player and Team Data******":"Column Name",
          "Unnamed: 1":"Description"}
explanations = explanations.rename(mapper=renamed, axis=1)
explanations

,Column Name,Description
25,playerId,Unique ID for each player assigned by the NHL
26,season,Starting year of the season. For example 2018 for the 2018-2019 season
27,situation,"5on5 for normal play, 5on4 for a normal powerplay, 4on5 for a normal PK. 'Other' includes everything else: two man advantage, empty net, 4on3, etc. 'all' includes all situations"
28,games_played,Number of games played.
29,icetime,Ice time in seconds
30,shifts,Number of shifts a player had
31,gameScore,Game Score rating as designed by @domluszczyszyn
32,onIce_xGoalsPercentage,On Ice xGoals For / (On Ice xGoals For + On Ice xGoals Against)
33,offIce_xGoalsPercentage,Off Ice xGoals For / (Off Ice xGoals For + Off Ice xGoals Against)
34,onIce_corsiPercentage,On Ice Shot Attempts For / (On Ice Shot Attempts For + On Ice Shot Attempts Against)


In [22]:
df.head(5)

,season,name,team,position,situation,games_played,icetime,shifts,gameScore,onIce_xGoalsPercentage,...,OffIce_F_xGoals,OffIce_A_xGoals,OffIce_F_shotAttempts,OffIce_A_shotAttempts,xGoalsForAfterShifts,xGoalsAgainstAfterShifts,corsiForAfterShifts,corsiAgainstAfterShifts,fenwickForAfterShifts,fenwickAgainstAfterShifts
playerId,,,,,,,,,,,,,,,,,,,,,
8479595,2017,Blake Hillman,CHI,D,other,4,70.0,2.0,-0.13,0.00,...,0.18,0.55,4.0,5.0,0.00,0.00,0.0,0.0,0.0,0.0
8479595,2017,Blake Hillman,CHI,D,all,4,4327.0,99.0,0.12,0.32,...,5.12,8.86,153.0,185.0,0.00,0.00,0.0,0.0,0.0,0.0
8479595,2017,Blake Hillman,CHI,D,5on5,4,3860.0,84.0,0.12,0.37,...,4.37,6.99,138.0,157.0,0.09,0.14,4.0,4.0,2.0,3.0
8479595,2017,Blake Hillman,CHI,D,4on5,4,392.0,12.0,0.12,0.02,...,0.01,1.06,1.0,17.0,0.00,0.00,0.0,0.0,0.0,0.0
8479595,2017,Blake Hillman,CHI,D,5on4,4,5.0,1.0,0.03,0.00,...,0.01,0.16,1.0,3.0,0.00,0.00,0.0,0.0,0.0,0.0


In [23]:
df.columns.tolist()

['season',
 'name',
 'team',
 'position',
 'situation',
 'games_played',
 'icetime',
 'shifts',
 'gameScore',
 'onIce_xGoalsPercentage',
 'offIce_xGoalsPercentage',
 'onIce_corsiPercentage',
 'offIce_corsiPercentage',
 'onIce_fenwickPercentage',
 'offIce_fenwickPercentage',
 'iceTimeRank',
 'I_F_xOnGoal',
 'I_F_xGoals',
 'I_F_xRebounds',
 'I_F_xFreeze',
 'I_F_xPlayStopped',
 'I_F_xPlayContinuedInZone',
 'I_F_xPlayContinuedOutsideZone',
 'I_F_flurryAdjustedxGoals',
 'I_F_scoreVenueAdjustedxGoals',
 'I_F_flurryScoreVenueAdjustedxGoals',
 'I_F_primaryAssists',
 'I_F_secondaryAssists',
 'I_F_shotsOnGoal',
 'I_F_missedShots',
 'I_F_blockedShotAttempts',
 'I_F_shotAttempts',
 'I_F_points',
 'I_F_goals',
 'I_F_rebounds',
 'I_F_reboundGoals',
 'I_F_freeze',
 'I_F_playStopped',
 'I_F_playContinuedInZone',
 'I_F_playContinuedOutsideZone',
 'I_F_savedShotsOnGoal',
 'I_F_savedUnblockedShotAttempts',
 'penalties',
 'I_F_penalityMinutes',
 'I_F_faceOffsWon',
 'I_F_hits',
 'I_F_takeaways',
 'I_F_giveaways',
 'I_F_lowDangerShots',
 'I_F_mediumDangerShots',
 'I_F_highDangerShots',
 'I_F_lowDangerxGoals',
 'I_F_mediumDangerxGoals',
 'I_F_highDangerxGoals',
 'I_F_lowDangerGoals',
 'I_F_mediumDangerGoals',
 'I_F_highDangerGoals',
 'I_F_scoreAdjustedShotsAttempts',
 'I_F_unblockedShotAttempts',
 'I_F_scoreAdjustedUnblockedShotAttempts',
 'I_F_dZoneGiveaways',
 'I_F_xGoalsFromxReboundsOfShots',
 'I_F_xGoalsFromActualReboundsOfShots',
 'I_F_reboundxGoals',
 'I_F_xGoals_with_earned_rebounds',
 'I_F_xGoals_with_earned_rebounds_scoreAdjusted',
 'I_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted',
 'I_F_shifts',
 'I_F_oZoneShiftStarts',
 'I_F_dZoneShiftStarts',
 'I_F_neutralZoneShiftStarts',
 'I_F_flyShiftStarts',
 'I_F_oZoneShiftEnds',
 'I_F_dZoneShiftEnds',
 'I_F_neutralZoneShiftEnds',
 'I_F_flyShiftEnds',
 'faceoffsWon',
 'faceoffsLost',
 'timeOnBench',
 'penalityMinutes',
 'penalityMinutesDrawn',
 'penaltiesDrawn',
 'shotsBlockedByPlayer',
 'OnIce_F_xOnGoal',
 'OnIce_F_xGoals',
 'OnIce_F_flurryAdjustedxGoals',
 'OnIce_F_scoreVenueAdjustedxGoals',
 'OnIce_F_flurryScoreVenueAdjustedxGoals',
 'OnIce_F_shotsOnGoal',
 'OnIce_F_missedShots',
 'OnIce_F_blockedShotAttempts',
 'OnIce_F_shotAttempts',
 'OnIce_F_goals',
 'OnIce_F_rebounds',
 'OnIce_F_reboundGoals',
 'OnIce_F_lowDangerShots',
 'OnIce_F_mediumDangerShots',
 'OnIce_F_highDangerShots',
 'OnIce_F_lowDangerxGoals',
 'OnIce_F_mediumDangerxGoals',
 'OnIce_F_highDangerxGoals',
 'OnIce_F_lowDangerGoals',
 'OnIce_F_mediumDangerGoals',
 'OnIce_F_highDangerGoals',
 'OnIce_F_scoreAdjustedShotsAttempts',
 'OnIce_F_unblockedShotAttempts',
 'OnIce_F_scoreAdjustedUnblockedShotAttempts',
 'OnIce_F_xGoalsFromxReboundsOfShots',
 'OnIce_F_xGoalsFromActualReboundsOfShots',
 'OnIce_F_reboundxGoals',
 'OnIce_F_xGoals_with_earned_rebounds',
 'OnIce_F_xGoals_with_earned_rebounds_scoreAdjusted',
 'OnIce_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted',
 'OnIce_A_xOnGoal',
 'OnIce_A_xGoals',
 'OnIce_A_flurryAdjustedxGoals',
 'OnIce_A_scoreVenueAdjustedxGoals',
 'OnIce_A_flurryScoreVenueAdjustedxGoals',
 'OnIce_A_shotsOnGoal',
 'OnIce_A_missedShots',
 'OnIce_A_blockedShotAttempts',
 'OnIce_A_shotAttempts',
 'OnIce_A_goals',
 'OnIce_A_rebounds',
 'OnIce_A_reboundGoals',
 'OnIce_A_lowDangerShots',
 'OnIce_A_mediumDangerShots',
 'OnIce_A_highDangerShots',
 'OnIce_A_lowDangerxGoals',
 'OnIce_A_mediumDangerxGoals',
 'OnIce_A_highDangerxGoals',
 'OnIce_A_lowDangerGoals',
 'OnIce_A_mediumDangerGoals',
 'OnIce_A_highDangerGoals',
 'OnIce_A_scoreAdjustedShotsAttempts',
 'OnIce_A_unblockedShotAttempts',
 'OnIce_A_scoreAdjustedUnblockedShotAttempts',
 'OnIce_A_xGoalsFromxReboundsOfShots',
 'OnIce_A_xGoalsFromActualReboundsOfShots',
 'OnIce_A_reboundxGoals',
 'OnIce_A_xGoals_with_earned_rebounds',
 'OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted',
 'OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted',
 'OffIce_F_xGoals',
 'OffIce_A_xGoals',
 'OffIce_F_shotAttempts',
 'OffIce_A_shotAttempts',
 'xGoalsForAfterShifts',
 'xGoa

</br> Tons of features.

In [24]:
df["position"].value_counts()

D    1530
C    1365
L     860
R     695
Name: position, dtype: int64

In [25]:
df["name"].value_counts()

Sebastian Aho               10
Denis Malgin                 5
Liam O'Brien                 5
Jake Dotchin                 5
Anton Stralman               5
Vladimir Sobotka             5
Josh Anderson                5
Morgan Rielly                5
Adrian Kempe                 5
Garnet Hathaway              5
Magnus Paajarvi              5
Christian Wolanin            5
Artemi Panarin               5
Curtis Lazar                 5
Damon Severson               5
Miles Wood                   5
Samuel Morin                 5
Brandon Saad                 5
Kerby Rychel                 5
Duncan Siemens               5
Curtis Valk                  5
Jordan Staal                 5
Auston Matthews              5
Daniel Catenacci             5
Andreas Johnsson             5
Frederick Gaudreau           5
Ryan Lomberg                 5
Henrik Borgstrom             5
Alex Chiasson                5
Austin Watson                5
Zach Whitecloud              5
Shawn Matthias               5
Justin W

</br>
There are some NHL players with identical names (e.g. Sebastion Aho). 

In [26]:
df_5on5 = df.loc[df["situation"] == "5on5"].drop(columns="situation")

df_all = df.loc[df["situation"] == "all"].drop(columns="situation")

df_other = df.loc[df["situation"] == "other"].drop(columns="situation")

df_4on5 = df.loc[df["situation"] == "4on5"].drop(columns="situation")

df_5on4 = df.loc[df["situation"] == "5on4"].drop(columns="situation")

In [27]:
#set back to default
pd.set_option('display.max_colwidth', 50, "display.max_rows", 60)

## Try to predict 5 on 5 goals

In [28]:
y = df_5on5[["I_F_goals"]]
y_primary_assists = df_5on5[["I_F_primaryAssists"]]
y_secondary_assists = df_5on5[["I_F_secondaryAssists"]]

In [29]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(df_5on5, y, test_size=0.2)

### Preprocessing

In [30]:
X_train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 8474849 to 8479465
Data columns (total 152 columns):
 #   Column                                                   Dtype  
---  ------                                                   -----  
 0   season                                                   int64  
 1   name                                                     object 
 2   team                                                     object 
 3   position                                                 object 
 4   games_played                                             int64  
 5   icetime                                                  float64
 6   shifts                                                   float64
 7   gameScore                                                float64
 8   onIce_xGoalsPercentage                                   float64
 9   offIce_xGoalsPercentage                                  float64
 10  onIce_corsiPercentage                  

*Important*:

The saved shots on goal feature must be dropped. The model scored extremely well on predicting goals, and upon further inspection, the largest positive coefficient identified by Ridge was shots on goal, while the largest negative coefficient was saved shots on goal. So the model essentially had learned something along the lines of goals = shots - saves shots.

In [31]:
drop_features = ["season", "name", "I_F_primaryAssists", "I_F_secondaryAssists",
               'I_F_points', 'I_F_goals', 'I_F_reboundGoals', 
               'I_F_lowDangerGoals', 'I_F_mediumDangerGoals', 'I_F_highDangerGoals', 
               'OnIce_F_goals', 'OnIce_F_reboundGoals',
               'OnIce_F_lowDangerGoals', 'OnIce_F_mediumDangerGoals', 'OnIce_F_highDangerGoals', 
                "I_F_savedShotsOnGoal"]

# get rid of all "on ice against" statistics
one_ice_a = []
for feat in X_train.columns.tolist():
    if feat[:7] == "OnIce_A":
        drop_features.append(feat) 
    

categorical_features = ["team", "position"]

numerical_features = X_train.columns.tolist()
for feat in drop_features + categorical_features: #everything else is numerical
    numerical_features.remove(feat)

In [32]:
preprocessor = make_column_transformer((StandardScaler(), numerical_features),
                                       (OneHotEncoder(handle_unknown = "ignore"), categorical_features),
                                       ("drop", drop_features))

new_categorical_features = []
for feat_name in X_train["team"].unique().tolist() + X_train["position"].unique().tolist():
    new_categorical_features.append(feat_name)

new_feature_names = numerical_features + new_categorical_features

### kNN

In [33]:
knn = KNeighborsClassifier(n_neighbors = 5)
pipeline = make_pipeline(preprocessor, knn)
cross_validate(pipeline, X_train, np.ravel(y_train), scoring="neg_mean_absolute_error")

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


{'fit_time': array([0.03901386, 0.03397417, 0.02638602, 0.02417397, 0.01559114]),
 'score_time': array([0.04253435, 0.05555987, 0.02757692, 0.02933288, 0.02782798]),
 'test_score': array([-2.12587413, -2.25874126, -2.28169014, -2.21830986, -2.48591549])}

## Predict powerplay goals

## Predict shorthanded goals

## Predict all goals

In [34]:
y_goals = df_all[["I_F_goals"]]

X_train, X_test, y_train, y_test = train_test_split(df_all, y_goals, test_size = 0.2)

In [35]:
pipe_17_18 = make_pipeline(preprocessor, Ridge())
cv_results = pd.DataFrame(cross_validate(pipe_17_18, X_train, np.ravel(y_train), scoring = "neg_mean_absolute_error"))
pd.DataFrame(cv_results.mean()).T

,fit_time,score_time,test_score
0,0.037709,0.011179,-0.990629


In [36]:
pipe_17_18.fit(X_train, np.ravel(y_train));

In [37]:
pipe_17_18.named_steps["ridge"].coef_

array([ 8.03086412e-01,  2.29023062e+00,  1.29959014e-01,  2.62289517e+00,
        2.26313998e-02,  6.71302788e-03,  1.93544569e-01,  6.41288256e-02,
       -2.66669371e-01, -1.03378470e-02, -2.14062882e-01,  2.05984785e+00,
        7.79454586e-01,  1.47893822e-01,  3.19245902e+00,  2.38266127e+00,
        6.20905134e+00,  3.79083377e+00,  5.63683461e-01,  1.00304323e+00,
        8.00200310e-01,  6.01748896e+00,  2.16352315e+00, -1.50640259e+00,
        3.31543329e+00, -2.72274004e+00, -6.93720674e+00, -1.64551454e+00,
       -1.30818585e+01, -1.46628799e+01, -1.19744720e+01, -1.54311177e-01,
        1.47232600e-01, -6.65552124e-02, -3.78719117e-02, -3.17453594e-01,
        2.86253745e-01,  5.91785231e+00,  1.97592563e+00, -6.18120317e-02,
        1.77547067e+00, -2.71699471e-01,  1.00241506e+00,  3.18612760e+00,
        4.99609600e+00,  4.98815849e+00, -1.61747285e-01,  1.14832212e+00,
       -1.37229183e-01,  8.99677625e-02,  4.41880243e-01,  7.74621655e-01,
        7.72615406e-01,  

In [38]:
X_train_transformed = pd.DataFrame(preprocessor.fit_transform(X_train), columns = new_feature_names)
X_train_transformed.columns.tolist()

['games_played',
 'icetime',
 'shifts',
 'gameScore',
 'onIce_xGoalsPercentage',
 'offIce_xGoalsPercentage',
 'onIce_corsiPercentage',
 'offIce_corsiPercentage',
 'onIce_fenwickPercentage',
 'offIce_fenwickPercentage',
 'iceTimeRank',
 'I_F_xOnGoal',
 'I_F_xGoals',
 'I_F_xRebounds',
 'I_F_xFreeze',
 'I_F_xPlayStopped',
 'I_F_xPlayContinuedInZone',
 'I_F_xPlayContinuedOutsideZone',
 'I_F_flurryAdjustedxGoals',
 'I_F_scoreVenueAdjustedxGoals',
 'I_F_flurryScoreVenueAdjustedxGoals',
 'I_F_shotsOnGoal',
 'I_F_missedShots',
 'I_F_blockedShotAttempts',
 'I_F_shotAttempts',
 'I_F_rebounds',
 'I_F_freeze',
 'I_F_playStopped',
 'I_F_playContinuedInZone',
 'I_F_playContinuedOutsideZone',
 'I_F_savedUnblockedShotAttempts',
 'penalties',
 'I_F_penalityMinutes',
 'I_F_faceOffsWon',
 'I_F_hits',
 'I_F_takeaways',
 'I_F_giveaways',
 'I_F_lowDangerShots',
 'I_F_mediumDangerShots',
 'I_F_highDangerShots',
 'I_F_lowDangerxGoals',
 'I_F_mediumDangerxGoals',
 'I_F_highDangerxGoals',
 'I_F_scoreAdjustedShotsAttempts',
 'I_F_unblockedShotAttempts',
 'I_F_scoreAdjustedUnblockedShotAttempts',
 'I_F_dZoneGiveaways',
 'I_F_xGoalsFromxReboundsOfShots',
 'I_F_xGoalsFromActualReboundsOfShots',
 'I_F_reboundxGoals',
 'I_F_xGoals_with_earned_rebounds',
 'I_F_xGoals_with_earned_rebounds_scoreAdjusted',
 'I_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted',
 'I_F_shifts',
 'I_F_oZoneShiftStarts',
 'I_F_dZoneShiftStarts',
 'I_F_neutralZoneShiftStarts',
 'I_F_flyShiftStarts',
 'I_F_oZoneShiftEnds',
 'I_F_dZoneShiftEnds',
 'I_F_neutralZoneShiftEnds',
 'I_F_flyShiftEnds',
 'faceoffsWon',
 'faceoffsLost',
 'timeOnBench',
 'penalityMinutes',
 'penalityMinutesDrawn',
 'penaltiesDrawn',
 'shotsBlockedByPlayer',
 'OnIce_F_xOnGoal',
 'OnIce_F_xGoals',
 'OnIce_F_flurryAdjustedxGoals',
 'OnIce_F_scoreVenueAdjustedxGoals',
 'OnIce_F_flurryScoreVenueAdjustedxGoals',
 'OnIce_F_shotsOnGoal',
 'OnIce_F_missedShots',
 'OnIce_F_blockedShotAttempts',
 'OnIce_F_shotAttempts',
 'OnIce_F_rebounds',
 'OnIce_F_lowDangerShots',
 'OnIce_F_mediumDangerShots',
 'OnIce_F_highDangerShots',
 'OnIce_F_lowDangerxGoals',
 'OnIce_F_mediumDangerxGoals',
 'OnIce_F_highDangerxGoals',
 'OnIce_F_scoreAdjustedShotsAttempts',
 'OnIce_F_unblockedShotAttempts',
 'OnIce_F_scoreAdjustedUnblockedShotAttempts',
 'OnIce_F_xGoalsFromxReboundsOfShots',
 'OnIce_F_xGoalsFromActualReboundsOfShots',
 'OnIce_F_reboundxGoals',
 'OnIce_F_xGoals_with_earned_rebounds',
 'OnIce_F_xGoals_with_earned_rebounds_scoreAdjusted',
 'OnIce_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted',
 'OffIce_F_xGoals',
 'OffIce_A_xGoals',
 'OffIce_F_shotAttempts',
 'OffIce_A_shotAttempts',
 'xGoalsForAfterShifts',
 'xGoalsAgainstAfterShifts',
 'corsiForAfterShifts',
 'corsiAgainstAfterShifts',
 'fenwickForAfterShifts',
 'fenwickAgainstAfterShifts',
 'DAL',
 'ARI',
 'PIT',
 'CHI',
 'CGY',
 'NSH',
 'VAN',
 'CAR',
 'N.J',
 'COL',
 'OTT',
 'BOS',
 'TOR',
 'NYI',
 'S.J',
 'L.A',
 'CBJ',
 'PHI',
 'FLA',
 'MTL',
 'EDM',
 'ANA',
 'WPG',
 'VGK',
 'BUF',
 'DET',
 'WSH',
 'STL',
 'MIN',
 'T.B',
 'NYR',
 'L',
 'D',
 'C',
 'R']

In [39]:
#Most important positive feature in 2017-2018 season
new_feature_names[np.argmax(pipe_17_18.named_steps["ridge"].coef_)]

'I_F_xPlayContinuedInZone'

In [40]:
#Most important negative feature in 2017-2018 season
new_feature_names[np.argmin(pipe_17_18.named_steps["ridge"].coef_)]

'I_F_playContinuedOutsideZone'

In [41]:
# List the features in order of deacreasing effect on goals
decreasing_importance = []
for i in reversed(np.argsort(pipe_17_18.named_steps["ridge"].coef_)):
    decreasing_importance.append(i)

sorted_features = []
for loc in decreasing_importance:
    sorted_features.append(new_feature_names[loc])
sorted_features

['I_F_xPlayContinuedInZone',
 'I_F_shotsOnGoal',
 'I_F_lowDangerShots',
 'I_F_unblockedShotAttempts',
 'I_F_scoreAdjustedUnblockedShotAttempts',
 'I_F_xPlayContinuedOutsideZone',
 'I_F_shotAttempts',
 'I_F_xFreeze',
 'I_F_scoreAdjustedShotsAttempts',
 'gameScore',
 'I_F_xPlayStopped',
 'icetime',
 'I_F_missedShots',
 'I_F_xOnGoal',
 'I_F_mediumDangerShots',
 'I_F_lowDangerxGoals',
 'I_F_xGoalsFromxReboundsOfShots',
 'I_F_neutralZoneShiftEnds',
 'I_F_scoreVenueAdjustedxGoals',
 'I_F_highDangerxGoals',
 'games_played',
 'I_F_flurryScoreVenueAdjustedxGoals',
 'I_F_xGoals',
 'I_F_xGoals_with_earned_rebounds_scoreAdjusted',
 'I_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted',
 'I_F_flurryAdjustedxGoals',
 'OnIce_F_shotsOnGoal',
 'VGK',
 'I_F_xGoals_with_earned_rebounds',
 'I_F_flyShiftStarts',
 'WSH',
 'OnIce_F_xGoalsFromxReboundsOfShots',
 'OffIce_A_shotAttempts',
 'OnIce_F_xGoalsFromActualReboundsOfShots',
 'FLA',
 'CAR',
 'I_F_giveaways',
 'OffIce_A_xGoals',
 'OnIce_F_mediumDangerxGoals',
 'EDM',
 'NYR',
 'STL',
 'NYI',
 'onIce_corsiPercentage',
 'shotsBlockedByPlayer',
 'BOS',
 'MTL',
 'ANA',
 'VAN',
 'PHI',
 'I_F_xRebounds',
 'I_F_penalityMinutes',
 'penalityMinutes',
 'C',
 'I_F_flyShiftEnds',
 'I_F_shifts',
 'shifts',
 'CBJ',
 'OnIce_F_lowDangerShots',
 'I_F_reboundxGoals',
 'timeOnBench',
 'I_F_oZoneShiftStarts',
 'OnIce_F_scoreVenueAdjustedxGoals',
 'offIce_corsiPercentage',
 'penaltiesDrawn',
 'OnIce_F_unblockedShotAttempts',
 'T.B',
 'D',
 'onIce_xGoalsPercentage',
 'MIN',
 'faceoffsLost',
 'OnIce_F_lowDangerxGoals',
 'offIce_xGoalsPercentage',
 'TOR',
 'xGoalsForAfterShifts',
 'fenwickAgainstAfterShifts',
 'fenwickForAfterShifts',
 'corsiAgainstAfterShifts',
 'corsiForAfterShifts',
 'xGoalsAgainstAfterShifts',
 'OnIce_F_xGoals',
 'offIce_fenwickPercentage',
 'OnIce_F_scoreAdjustedUnblockedShotAttempts',
 'I_F_hits',
 'DET',
 'N.J',
 'CHI',
 'L',
 'I_F_highDangerShots',
 'I_F_faceOffsWon',
 'faceoffsWon',
 'OnIce_F_highDangerShots',
 'OnIce_F_reboundxGoals',
 'R',
 'DAL',
 'OnIce_F_xGoals_with_earned_rebounds_scoreAdjusted',
 'NSH',
 'I_F_xGoalsFromActualReboundsOfShots',
 'OnIce_F_rebounds',
 'WPG',
 'penalties',
 'I_F_dZoneGiveaways',
 'OnIce_F_mediumDangerShots',
 'ARI',
 'iceTimeRank',
 'PIT',
 'OnIce_F_shotAttempts',
 'I_F_dZoneShiftStarts',
 'OnIce_F_xGoals_with_earned_rebounds',
 'OnIce_F_highDangerxGoals',
 'onIce_fenwickPercentage',
 'I_F_mediumDangerxGoals',
 'BUF',
 'penalityMinutesDrawn',
 'L.A',
 'I_F_takeaways',
 'S.J',
 'OnIce_F_xOnGoal',
 'OTT',
 'I_F_oZoneShiftEnds',
 'OnIce_F_scoreAdjustedShotsAttempts',
 'I_F_neutralZoneShiftStarts',
 'OffIce_F_xGoals',
 'OnIce_F_flurryScoreVenueAdjustedxGoals',
 'CGY',
 'I_F_dZoneShiftEnds',
 'OnIce_F_flurryAdjustedxGoals',
 'OnIce_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted',
 'COL',
 'OffIce_F_shotAttempts',
 'OnIce_F_missedShots',
 'OnIce_F_blockedShotAttempts',
 'I_F_blockedShotAttempts',
 'I_F_playStopped',
 'I_F_rebounds',
 'I_F_freeze',
 'I_F_savedUnblockedShotAttempts',
 'I_F_playContinuedInZone',
 'I_F_playContinuedOutsideZone']

In [42]:
pipe_17_18.fit(X_train, np.ravel(y_train))
y_pred = pipe_17_18.predict(X_test)
mean_absolute_error(np.ravel(y_test), y_pred)

0.8528128774884859

In [43]:
differences = np.ravel(y_test) - y_pred # Positive means overperformance
np.argsort(differences) # Earlier in the list means more underperforming

array([ 98, 166,  84,  51, 171,  92,  72, 125, 111, 151,  42,  59,   1,
       160,  47, 117,  90,  21,  12, 161, 120, 150, 130,  78,  76,  97,
        36,  57, 110, 173,  20,  43,  15, 146,  68,  11,  95,  91,  35,
       114, 102,  14,  61, 177, 133,  96,  34,  17, 112,  94, 162, 165,
       123,  19,  16,  58, 154,  66, 163,  62, 140, 143,   8,  86,  99,
       109,  32,  93,  89, 169, 145,  13,  54, 142,  24, 175,  88,  41,
         6, 131, 116,  46,   4,  80, 167,  75, 159, 103,  73,   3,  40,
       172, 149,   9, 170,  37,  64, 141, 119,  69,   0,  31, 158, 174,
       152,  71,  49,  55, 155, 124, 168,  52,  56,  82, 148, 122,  38,
       134, 126,  27, 107, 147,  33, 138,  60, 132, 144, 176,   2, 101,
       106,  53, 164,  70, 139,  18, 127,  83,  79,  39, 137, 115,  67,
        65,  25, 153, 113,  22, 136, 100,  87,  29, 104, 156,  30,  50,
       157, 108,  85,   5,  26,  77,  23, 118,  63,  44,  74, 135,   7,
       129, 121,  10, 105,  45, 128,  28,  81,  48])

In [44]:
location = 75

In [45]:
pd.DataFrame(y_test.iloc[location]).T

,I_F_goals
8474709,3.0


In [46]:
location = pd.DataFrame(y_test.iloc[location]).T.index[0]
df.loc[location]

,season,name,team,position,situation,games_played,icetime,shifts,gameScore,onIce_xGoalsPercentage,...,OffIce_F_xGoals,OffIce_A_xGoals,OffIce_F_shotAttempts,OffIce_A_shotAttempts,xGoalsForAfterShifts,xGoalsAgainstAfterShifts,corsiForAfterShifts,corsiAgainstAfterShifts,fenwickForAfterShifts,fenwickAgainstAfterShifts
playerId,,,,,,,,,,,,,,,,,,,,,
8474709,2017,Matt Martin,TOR,L,other,50,316.0,5.0,-1.31,0.37,...,4.21,5.33,24.0,37.0,0.0,0.16,0.0,1.0,0.0,1.0
8474709,2017,Matt Martin,TOR,L,all,50,23965.0,604.0,10.28,0.48,...,150.29,132.87,2659.0,2631.0,0.0,0.00,0.0,0.0,0.0,0.0
8474709,2017,Matt Martin,TOR,L,5on5,50,23595.0,592.0,10.28,0.48,...,94.01,84.08,2038.0,1990.0,0.6,1.03,15.0,22.0,13.0,19.0
8474709,2017,Matt Martin,TOR,L,4on5,50,0.0,0.0,0.00,0.00,...,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
8474709,2017,Matt Martin,TOR,L,5on4,50,54.0,7.0,1.10,0.00,...,8.89,0.37,90.0,11.0,0.0,0.00,0.0,0.0,0.0,0.0


## Predicting next year's goal totals

In [47]:
df2 = pd.read_csv("Players_18_19.csv", index_col="playerId")
df2_all = df2.loc[df2["situation"] == "all"].drop(columns="situation")

In [48]:
X_next_year = df2_all#.drop(columns="I_F_goals")
y_next_year = df2_all[["I_F_goals"]]

In [49]:
X_train_next, X_test_next, y_train_next, y_test_next = train_test_split(X_next_year, y_next_year, test_size=0.2)

In [50]:
pipe_18_19 = make_pipeline(preprocessor, Ridge())

In [51]:
pipe_17_18.fit(X_train, np.ravel(y_train))

pipe_18_19.fit(X_train_next, np.ravel(y_train_next));

In [52]:
# Calculate MAE for predictions of next year's goal totals using 
# this year's data, but last year's model
y_pred_next = pipe_17_18.predict(X_next_year)
mean_absolute_error(np.ravel(y_next_year), y_pred_next)

1.0462136374879722

In [53]:
# How good are the predcitions for next year?
differences = np.ravel(y_next_year) - y_pred_next # Positive means overperformance in 2018-2019
for diff in np.sort(differences):
    print(diff) # Earlier in the list means more underperforming in 2018-2019

-4.2952545957470605
-4.206989522838297
-3.8263518688512113
-3.818804569904527
-3.2402912481369057
-3.146732258163598
-3.1111992653695903
-3.0775038128903986
-2.916850486288226
-2.881974416742217
-2.8489898508144336
-2.7989633630088804
-2.7814578026151624
-2.740606191074898
-2.7386571346852513
-2.6896376820123447
-2.68772600240521
-2.6740574166929214
-2.6711109901527657
-2.57926609174796
-2.573463855620016
-2.571659255234497
-2.566701586190778
-2.548289598679048
-2.536820260601406
-2.4051415373476246
-2.3596427753754856
-2.3576466206836955
-2.3006843246763893
-2.2957785246174804
-2.2947915868877047
-2.270366612442942
-2.2432260647929656
-2.237474739748402
-2.2189318740661603
-2.2187020882970216
-2.206887809951464
-2.183841618428545
-2.136697523546058
-2.1335122169974845
-2.098543460173394
-2.096486935621095
-2.071182811320874
-2.0711109307446396
-2.0494294450384736
-2.045215248528166
-2.0394606545994236
-2.014314111111327
-1.9793680846912904
-1.9293684694260573
-1.8964897793398787
-1.89

In [54]:
# standard deviation. Might be interesting to see all the players falling
# 2 std devs from the mean predicted score and what percent of them actually
# over/underperformed based on next year's results.
np.std(differences)

1.3052247202515441

In [55]:
np.argsort(differences) # Earlier in the list means more underperforming

array([ 78, 405, 706, 163, 572, 486,  91, 632,  30, 785, 111, 797, 357,
       135, 188,  88, 661, 595,  99, 115, 701, 634, 119, 196,  41, 330,
       604, 833, 228, 210, 846, 282,  47,  22, 206, 437, 787, 380, 726,
        16, 249, 766, 848, 885, 589, 687, 311, 771, 490, 226, 635, 204,
        23, 519, 182, 517, 755, 628, 558, 659,  29, 727, 116, 495, 475,
       407,  25, 666, 720,  17, 745, 645, 489, 831, 509, 579,  60, 459,
       893, 265, 390, 410, 417, 190, 538, 524, 103, 568, 775, 681, 587,
       547, 377, 472, 734, 444, 260, 503, 343,  70, 349, 448, 655, 690,
        90, 637, 867, 167, 777, 320, 213, 725, 552, 744, 626,  33, 657,
        63, 422, 131, 181, 394, 854, 715, 385, 125, 133, 501, 224, 194,
       602, 205, 767, 748, 789, 742, 324, 829,  32, 481, 379, 360, 618,
       580, 852, 556, 750, 710, 209, 348, 783, 461, 253,  27, 446, 737,
       283, 606, 823,   2, 549, 786, 860, 798, 813, 492, 439, 640, 339,
       825, 616, 445, 429, 515, 551, 316, 559,  72, 174, 364, 87

In [56]:
n = 10
N = len(differences)

print("Underperformers in 2018-2019")
print("------------------------------")
for i in range(n):
    
    # Underperformers
    j = np.argsort(differences)[i]
    player_id = pd.DataFrame(y_next_year.iloc[j]).T.index[0]
    
    name = pd.DataFrame(df_all.loc[player_id]).T["name"].iloc[0]
    goals_17_18 = pd.DataFrame(df_all.loc[player_id]).T["I_F_goals"].iloc[0]
    goals_18_19 = pd.DataFrame(df2_all.loc[player_id]).T["I_F_goals"].iloc[0]
    pred_goals_17_18 = pipe_17_18.predict(pd.DataFrame(df_all.loc[player_id]).T)
    pred_goals_18_19 = pipe_18_19.predict(pd.DataFrame(df2_all.loc[player_id]).T)
    
    print("Player:", name)
    print("Player ID:", player_id)
    print("Goals in 2017-2018:", goals_17_18)
    print("Goals in 2018-2019:", goals_18_19)
    print("Predicted goals in 2017-2018: {:.2f}".format(float(pred_goals_17_18)))
    print("Predicted goals in 2018-2019: {:.2f}".format(float(pred_goals_18_19)))
    print("\n")
    
print("\n\n")
print("Overperformers in 2018-2019")
print("------------------------------")

for i in range(n):
    
    # Overperformers
    j = np.argsort(differences)[N-i-1]
    player_id = pd.DataFrame(y_next_year.iloc[j]).T.index[0]
    
    name = pd.DataFrame(df_all.loc[player_id]).T["name"].iloc[0]
    goals_17_18 = pd.DataFrame(df_all.loc[player_id]).T["I_F_goals"].iloc[0]
    goals_18_19 = pd.DataFrame(df2_all.loc[player_id]).T["I_F_goals"].iloc[0]
    pred_goals_17_18 = pipe_17_18.predict(pd.DataFrame(df_all.loc[player_id]).T)
    pred_goals_18_19 = pipe_18_19.predict(pd.DataFrame(df2_all.loc[player_id]).T)


    print("Player:", name)
    print("Player ID:", player_id)
    print("Goals in 2017-2018:", goals_17_18)
    print("Goals in 2018-2019:", goals_18_19)
    print("Predicted goals in 2017-2018: {:.2f}".format(float(pred_goals_17_18)))
    print("Predicted goals in 2018-2019: {:.2f}".format(float(pred_goals_18_19)))
    print("\n")

Underperformers in 2018-2019
------------------------------
Player: Brayden Schenn
Player ID: 8475170
Goals in 2017-2018: 28.0
Goals in 2018-2019: 17.0
Predicted goals in 2017-2018: 27.37
Predicted goals in 2018-2019: 20.75


Player: Jaden Schwartz
Player ID: 8475768
Goals in 2017-2018: 24.0
Goals in 2018-2019: 11.0
Predicted goals in 2017-2018: 23.98
Predicted goals in 2018-2019: 14.34


Player: Tobias Rieder
Player ID: 8476356
Goals in 2017-2018: 12.0
Goals in 2018-2019: 0.0
Predicted goals in 2017-2018: 11.47
Predicted goals in 2018-2019: 3.64


Player: Cal Clutterbuck
Player ID: 8473504
Goals in 2017-2018: 8.0
Goals in 2018-2019: 8.0
Predicted goals in 2017-2018: 7.96
Predicted goals in 2018-2019: 10.37


Player: Kevin Hayes
Player ID: 8475763
Goals in 2017-2018: 25.0
Goals in 2018-2019: 19.0
Predicted goals in 2017-2018: 21.71
Predicted goals in 2018-2019: 20.93


Player: Matt Nieto
Player ID: 8476442
Goals in 2017-2018: 15.0
Goals in 2018-2019: 3.0
Predicted goals in 2017-2018: 1

## Find Over/Underperformers in 2017-2018 and Compare to 2018-2019

In [57]:
# Make a prediction on the test set in 2017-2018
y_pred_17_18 = pipe_17_18.predict(X_test)

# See the biggest outliers
differences_17_18 = np.ravel(y_test) - y_pred_17_18

n = 10
N = len(differences_17_18)

print("Underperformers in 2017-2018")
print("------------------------------")
for i in range(n):
    
    # Underperformers
    j = np.argsort(differences_17_18)[i]
    player_id = pd.DataFrame(y_test.iloc[j]).T.index[0]
    
    name = pd.DataFrame(df_all.loc[player_id]).T["name"].iloc[0]
    
    # Make sure that player actually played next year
    if player_id in df2_all.index.tolist():
        
        goals_17_18 = pd.DataFrame(df_all.loc[player_id]).T["I_F_goals"].iloc[0]
        goals_18_19 = pd.DataFrame(df2_all.loc[player_id]).T["I_F_goals"].iloc[0]
        pred_goals_17_18 = pipe_17_18.predict(pd.DataFrame(df_all.loc[player_id]).T)
        pred_goals_18_19 = pipe_18_19.predict(pd.DataFrame(df2_all.loc[player_id]).T)

        print("Player:", name)
        print("Player ID:", player_id)
        print("Goals in 2017-2018:", goals_17_18)
        print("Predicted goals in 2017-2018: {:.2f}".format(float(pred_goals_17_18)))

        print("Goals in 2018-2019:", goals_18_19)
        print("Predicted goals in 2018-2019: {:.2f}".format(float(pred_goals_18_19)))
        print("\n")
        
    else:
        
        print("Player", name, "did not play in the 2018-2019 season.\n\n")

print("\n\n")
print("Overperformers in 2017-2018")
print("------------------------------")

for i in range(n):
    
    # Overperformers
    j = np.argsort(differences_17_18)[N-i-1]
    player_id = pd.DataFrame(y_test.iloc[j]).T.index[0]
    
    name = pd.DataFrame(df_all.loc[player_id]).T["name"].iloc[0]
    
    # Make sure that player actually played next year
    if player_id in df2_all.index.tolist():
        
        goals_17_18 = pd.DataFrame(df_all.loc[player_id]).T["I_F_goals"].iloc[0]
        goals_18_19 = pd.DataFrame(df2_all.loc[player_id]).T["I_F_goals"].iloc[0]
        pred_goals_17_18 = pipe_17_18.predict(pd.DataFrame(df_all.loc[player_id]).T)
        pred_goals_18_19 = pipe_18_19.predict(pd.DataFrame(df2_all.loc[player_id]).T)


        print("Player:", name)
        print("Player ID:", player_id)
        print("Goals in 2017-2018:", goals_17_18)
        print("Predicted goals in 2017-2018: {:.2f}".format(float(pred_goals_17_18)))
        
        print("Goals in 2018-2019:", goals_18_19)
        print("Predicted goals in 2018-2019: {:.2f}".format(float(pred_goals_18_19)))
        print("\n")
        
    else:
        
        print("Player", name, "did not play in the 2018-2019 season.\n\n")

Underperformers in 2017-2018
------------------------------
Player: Jakub Voracek
Player ID: 8474161
Goals in 2017-2018: 20.0
Predicted goals in 2017-2018: 24.15
Goals in 2018-2019: 21.0
Predicted goals in 2018-2019: 21.22


Player: Paul Stastny
Player ID: 8471669
Goals in 2017-2018: 16.0
Predicted goals in 2017-2018: 19.15
Goals in 2018-2019: 13.0
Predicted goals in 2018-2019: 13.89


Player: Milan Lucic
Player ID: 8473473
Goals in 2017-2018: 10.0
Predicted goals in 2017-2018: 12.40
Goals in 2018-2019: 6.0
Predicted goals in 2018-2019: 5.35


Player: Alexander Steen
Player ID: 8470257
Goals in 2017-2018: 15.0
Predicted goals in 2017-2018: 17.34
Goals in 2018-2019: 11.0
Predicted goals in 2018-2019: 10.93


Player: Andrew Ladd
Player ID: 8471217
Goals in 2017-2018: 12.0
Predicted goals in 2017-2018: 14.11
Goals in 2018-2019: 3.0
Predicted goals in 2018-2019: 2.73


Player: Kailer Yamamoto
Player ID: 8479977
Goals in 2017-2018: 0.0
Predicted goals in 2017-2018: 2.05
Goals in 2018-2019: 

## Same But Using Goals per Game

In [58]:
# Make a prediction on the test set in 2017-2018
y_pred_17_18 = pipe_17_18.predict(X_test)

# See the biggest outliers
differences_17_18 = np.ravel(y_test) - y_pred_17_18

n = 10
N = len(differences_17_18)

print("Underperformers in 2017-2018")
print("------------------------------")
for i in range(n):
    
    print("------------------------------")
    print("\n")
    
    # Underperformers
    j = np.argsort(differences_17_18)[i]
    player_id = pd.DataFrame(y_test.iloc[j]).T.index[0]
    
    name = pd.DataFrame(df_all.loc[player_id]).T["name"].iloc[0]
    
    # Make sure that player actually played next year
    if player_id in df2_all.index.tolist():
        
        games_played_17_18 = df_all["games_played"].loc[player_id]
        games_played_18_19 = df2_all["games_played"].loc[player_id]
        
        goals_17_18 = pd.DataFrame(df_all.loc[player_id]).T["I_F_goals"].iloc[0]
        goals_18_19 = pd.DataFrame(df2_all.loc[player_id]).T["I_F_goals"].iloc[0]
        pred_goals_17_18 = pipe_17_18.predict(pd.DataFrame(df_all.loc[player_id]).T)[0]
        pred_goals_18_19 = pipe_18_19.predict(pd.DataFrame(df2_all.loc[player_id]).T)[0]

        print("Player:", name)
        print("Player ID:", player_id)
        print("\n")
        
        print("2017-2018 Season")
        print("----------------------")
        print("Goal pace over 82 games: {:.2f}".format(goals_17_18 / games_played_17_18 * 82))
        print("Predicted goal pace over 82 games: {:.2f}".format(pred_goals_17_18 / games_played_17_18 * 82))
        print("Games played:", games_played_17_18)
        print("\n")

        print("2018-2019 Season")
        print("----------------------")
        print("Goal pace over 82 games: {:.2f}".format(goals_18_19 / games_played_18_19 * 82))
        print("Predicted goal pace over 82 games: {:.2f}".format(pred_goals_18_19 / games_played_18_19 * 82))
        print("Games played:", games_played_18_19)
        print("\n")
        
    else:
        
        print("Player", name, "did not play in the 2018-2019 season.\n\n")

print("\n\n")
print("Overperformers in 2017-2018")
print("------------------------------")

for i in range(n):
    
    print("------------------------------")
    print("\n")
    
    # Overperformers
    j = np.argsort(differences_17_18)[N-i-1]
    player_id = pd.DataFrame(y_test.iloc[j]).T.index[0]
    
    name = pd.DataFrame(df_all.loc[player_id]).T["name"].iloc[0]
    
    # Make sure that player actually played next year
    if player_id in df2_all.index.tolist():
        
        games_played_17_18 = df_all["games_played"].loc[player_id]
        games_played_18_19 = df2_all["games_played"].loc[player_id]
        
        goals_17_18 = pd.DataFrame(df_all.loc[player_id]).T["I_F_goals"].iloc[0]
        goals_18_19 = pd.DataFrame(df2_all.loc[player_id]).T["I_F_goals"].iloc[0]
        pred_goals_17_18 = pipe_17_18.predict(pd.DataFrame(df_all.loc[player_id]).T)[0]
        pred_goals_18_19 = pipe_18_19.predict(pd.DataFrame(df2_all.loc[player_id]).T)[0]


        print("Player:", name)
        print("Player ID:", player_id)
        print("\n")
        
        print("2017-2018 Season")
        print("----------------------")
        
        print("Goal pace over 82 games: {:.2f}".format(goals_17_18 / games_played_17_18 * 82))
        print("Predicted goal pace over 82 games: {:.2f}".format(pred_goals_17_18 / games_played_17_18 * 82))
        print("Games played:", games_played_17_18)
        print("\n")

        print("2018-2019 Season")
        print("----------------------")
        print("Goal pace over 82 games: {:.2f}".format(goals_18_19 / games_played_18_19 * 82))
        print("Predicted goal pace over 82 games: {:.2f}".format(pred_goals_18_19 / games_played_18_19 * 82))
        print("Games played:", games_played_18_19)
        print("\n")
        
    else:
        
        print("Player", name, "did not play in the 2018-2019 season.\n\n")

Underperformers in 2017-2018
------------------------------
------------------------------


Player: Jakub Voracek
Player ID: 8474161


2017-2018 Season
----------------------
Goal pace over 82 games: 20.00
Predicted goal pace over 82 games: 24.15
Games played: 82


2018-2019 Season
----------------------
Goal pace over 82 games: 22.08
Predicted goal pace over 82 games: 22.30
Games played: 78


------------------------------


Player: Paul Stastny
Player ID: 8471669


2017-2018 Season
----------------------
Goal pace over 82 games: 16.00
Predicted goal pace over 82 games: 19.15
Games played: 82


2018-2019 Season
----------------------
Goal pace over 82 games: 21.32
Predicted goal pace over 82 games: 22.77
Games played: 50


------------------------------


Player: Milan Lucic
Player ID: 8473473


2017-2018 Season
----------------------
Goal pace over 82 games: 10.00
Predicted goal pace over 82 games: 12.40
Games played: 82


2018-2019 Season
----------------------
Goal pace over 82 ga

## Using Lasso

In [59]:
pipe_17_18_Lasso = make_pipeline(preprocessor, Lasso(alpha=0.25))
pipe_18_19_Lasso = make_pipeline(preprocessor, Lasso(alpha=0.25))

In [60]:
pipe_17_18_Lasso.fit(X_train, np.ravel(y_train))
pipe_18_19_Lasso.fit(X_train_next, np.ravel(y_train_next));

In [61]:
lasso_coefs = pipe_17_18_Lasso.named_steps["lasso"].coef_
lasso_coefs

array([-0.        , -0.        , -0.        ,  2.23879671, -0.        ,
       -0.        , -0.        , -0.18333029, -0.        , -0.        ,
       -0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -0.        , -0.        ,  0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.08342849,
       -0.        , -0.        , -0.        ,  0.59154273,  0.        ,
        0.        ,  0.        ,  1.90583964,  0.        ,  0.        ,
        0.        , -0.09744855,  0.        , -0.        ,  0.        ,
        0.        ,  1.00476506,  3.19380006, -0.        ,  0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        ,  0.        , -0.01161333, -0.  

In [62]:
for i in range(len(lasso_coefs)):
    if lasso_coefs[i] != 0:
        print("Feature:", new_feature_names[i])
        print("Coefficient value:", lasso_coefs[i])
        print("-----------------------------------------------")

Feature: gameScore
Coefficient value: 2.238796709520037
-----------------------------------------------
Feature: offIce_corsiPercentage
Coefficient value: -0.18333028883034133
-----------------------------------------------
Feature: I_F_hits
Coefficient value: -0.08342848737249692
-----------------------------------------------
Feature: I_F_mediumDangerShots
Coefficient value: 0.5915427337586271
-----------------------------------------------
Feature: I_F_highDangerxGoals
Coefficient value: 1.9058396432710074
-----------------------------------------------
Feature: I_F_dZoneGiveaways
Coefficient value: -0.09744854667724036
-----------------------------------------------
Feature: I_F_xGoals_with_earned_rebounds_scoreAdjusted
Coefficient value: 1.0047650630644909
-----------------------------------------------
Feature: I_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted
Coefficient value: 3.193800063278025
-----------------------------------------------
Feature: shotsBlockedByPlayer
Coef

In [63]:
# Make a prediction on the test set in 2017-2018
y_pred_17_18 = pipe_17_18_Lasso.predict(X_test)

# See the biggest outliers
differences_17_18 = np.ravel(y_test) - y_pred_17_18

n = 10
N = len(differences_17_18)

print("Underperformers in 2017-2018")
print("------------------------------")
for i in range(n):
    
    print("------------------------------")
    print("\n")
    
    # Underperformers
    j = np.argsort(differences_17_18)[i]
    player_id = pd.DataFrame(y_test.iloc[j]).T.index[0]
    
    name = pd.DataFrame(df_all.loc[player_id]).T["name"].iloc[0]
    
    # Make sure that player actually played next year
    if player_id in df2_all.index.tolist():
        
        games_played_17_18 = df_all["games_played"].loc[player_id]
        games_played_18_19 = df2_all["games_played"].loc[player_id]
        
        goals_17_18 = pd.DataFrame(df_all.loc[player_id]).T["I_F_goals"].iloc[0]
        goals_18_19 = pd.DataFrame(df2_all.loc[player_id]).T["I_F_goals"].iloc[0]
        pred_goals_17_18 = pipe_17_18_Lasso.predict(pd.DataFrame(df_all.loc[player_id]).T)[0]
        pred_goals_18_19 = pipe_18_19_Lasso.predict(pd.DataFrame(df2_all.loc[player_id]).T)[0]

        print("Player:", name)
        print("Player ID:", player_id)
        print("\n")
        
        print("2017-2018 Season")
        print("----------------------")
        print("Goal pace over 82 games: {:.2f}".format(goals_17_18 / games_played_17_18 * 82))
        print("Predicted goal pace over 82 games: {:.2f}".format(pred_goals_17_18 / games_played_17_18 * 82))
        print("Games played:", games_played_17_18)
        print("\n")

        print("2018-2019 Season")
        print("----------------------")
        print("Goal pace over 82 games: {:.2f}".format(goals_18_19 / games_played_18_19 * 82))
        print("Predicted goal pace over 82 games: {:.2f}".format(pred_goals_18_19 / games_played_18_19 * 82))
        print("Games played:", games_played_18_19)
        print("\n")
        
    else:
        
        print("Player", name, "did not play in the 2018-2019 season.\n\n")

print("\n\n")
print("Overperformers in 2017-2018")
print("------------------------------")

for i in range(n):
    
    print("------------------------------")
    print("\n")
    
    # Overperformers
    j = np.argsort(differences_17_18)[N-i-1]
    player_id = pd.DataFrame(y_test.iloc[j]).T.index[0]
    
    name = pd.DataFrame(df_all.loc[player_id]).T["name"].iloc[0]
    
    # Make sure that player actually played next year
    if player_id in df2_all.index.tolist():
        
        games_played_17_18 = df_all["games_played"].loc[player_id]
        games_played_18_19 = df2_all["games_played"].loc[player_id]
        
        goals_17_18 = pd.DataFrame(df_all.loc[player_id]).T["I_F_goals"].iloc[0]
        goals_18_19 = pd.DataFrame(df2_all.loc[player_id]).T["I_F_goals"].iloc[0]
        pred_goals_17_18 = pipe_17_18_Lasso.predict(pd.DataFrame(df_all.loc[player_id]).T)[0]
        pred_goals_18_19 = pipe_18_19_Lasso.predict(pd.DataFrame(df2_all.loc[player_id]).T)[0]


        print("Player:", name)
        print("Player ID:", player_id)
        print("\n")
        
        print("2017-2018 Season")
        print("----------------------")
        
        print("Goal pace over 82 games: {:.2f}".format(goals_17_18 / games_played_17_18 * 82))
        print("Predicted goal pace over 82 games: {:.2f}".format(pred_goals_17_18 / games_played_17_18 * 82))
        print("Games played:", games_played_17_18)
        print("\n")

        print("2018-2019 Season")
        print("----------------------")
        print("Goal pace over 82 games: {:.2f}".format(goals_18_19 / games_played_18_19 * 82))
        print("Predicted goal pace over 82 games: {:.2f}".format(pred_goals_18_19 / games_played_18_19 * 82))
        print("Games played:", games_played_18_19)
        print("\n")
        
    else:
        
        print("Player", name, "did not play in the 2018-2019 season.\n\n")

Underperformers in 2017-2018
------------------------------
------------------------------


Player: Milan Lucic
Player ID: 8473473


2017-2018 Season
----------------------
Goal pace over 82 games: 10.00
Predicted goal pace over 82 games: 18.05
Games played: 82


2018-2019 Season
----------------------
Goal pace over 82 games: 6.23
Predicted goal pace over 82 games: 9.37
Games played: 79


------------------------------


Player: Jakub Voracek
Player ID: 8474161


2017-2018 Season
----------------------
Goal pace over 82 games: 20.00
Predicted goal pace over 82 games: 27.40
Games played: 82


2018-2019 Season
----------------------
Goal pace over 82 games: 22.08
Predicted goal pace over 82 games: 25.13
Games played: 78


------------------------------


Player: Brandon Saad
Player ID: 8476438


2017-2018 Season
----------------------
Goal pace over 82 games: 18.00
Predicted goal pace over 82 games: 24.90
Games played: 82


2018-2019 Season
----------------------
Goal pace over 82 game

In [64]:
# Pair 2017-2018 data with 2018-2019 goal totals by deleting
# players which don't show up in both X_train for 2017-2018 and y_train for 2018-2019
# Also drop players who plad less than 30 games in either season.
X_train_paired = X_train
X_test_paired = X_test
y_train_next_paired = y_train_next
y_test_next_paired = y_test_next
min_games = 30

for ind in X_train.index.tolist():
    
    if ind not in y_train_next.index.tolist():
        
        X_train_paired = X_train_paired.drop(ind)
        
    elif X_train.loc[ind]["games_played"] < 30:
        
        X_train_paired = X_train_paired.drop(ind)
        y_train_next_paired = y_train_next_paired.drop(ind)
        
    elif X_train_next.loc[ind]["games_played"] < 30:
        
        X_train_paired = X_train_paired.drop(ind)
        y_train_next_paired = y_train_next_paired.drop(ind)
        
for ind in y_train_next.index.tolist():
    if ind not in X_train.index.tolist():
        y_train_next_paired = y_train_next_paired.drop(ind)
        
for ind in X_test.index.tolist():
    
    if ind not in y_test_next.index.tolist():
        
        X_test_paired = X_test_paired.drop(ind)
        
    elif X_test.loc[ind]["games_played"] < 30:
        
        X_test_paired = X_test_paired.drop(ind)
        y_test_next_paired = y_test_next_paired.drop(ind)
        
    elif X_test_next.loc[ind]["games_played"] < 30:
        
        X_test_paired = X_test_paired.drop(ind)
        y_test_next_paired = y_test_next_paired.drop(ind)
        
for ind in y_test_next.index.tolist():
    if ind not in X_test.index.tolist():
        y_test_next_paired = y_test_next_paired.drop(ind)

In [65]:
pipe_17_18_Lasso = make_pipeline(preprocessor, Lasso(alpha=0.2))
pipe_17_18_Lasso.fit(X_train_paired, np.ravel(y_train_next_paired));

In [66]:
# Make a prediction on the test set in 2017-2018
y_pred_17_18 = pipe_17_18_Lasso.predict(X_test)

# See the biggest outliers
differences_17_18 = np.ravel(y_test) - y_pred_17_18

n = 10
N = len(differences_17_18)

print("Underperformers in 2017-2018")
print("------------------------------")
for i in range(n):
    
    print("------------------------------")
    print("\n")
    
    # Underperformers
    j = np.argsort(differences_17_18)[i]
    player_id = pd.DataFrame(y_test.iloc[j]).T.index[0]
    
    name = pd.DataFrame(df_all.loc[player_id]).T["name"].iloc[0]
    
    # Make sure that player actually played next year
    if player_id in df2_all.index.tolist():
        
        games_played_17_18 = df_all["games_played"].loc[player_id]
        games_played_18_19 = df2_all["games_played"].loc[player_id]
        
        goals_17_18 = pd.DataFrame(df_all.loc[player_id]).T["I_F_goals"].iloc[0]
        goals_18_19 = pd.DataFrame(df2_all.loc[player_id]).T["I_F_goals"].iloc[0]
        pred_goals_17_18 = pipe_17_18_Lasso.predict(pd.DataFrame(df_all.loc[player_id]).T)[0]
        pred_goals_18_19 = pipe_18_19_Lasso.predict(pd.DataFrame(df2_all.loc[player_id]).T)[0]

        print("Player:", name)
        print("Player ID:", player_id)
        print("\n")
        
        print("2017-2018 Season")
        print("----------------------")
        print("Goal pace over 82 games: {:.2f}".format(goals_17_18 / games_played_17_18 * 82))
        print("Predicted goal pace over 82 games: {:.2f}".format(pred_goals_17_18 / games_played_17_18 * 82))
        print("Games played:", games_played_17_18)
        print("\n")

        print("2018-2019 Season")
        print("----------------------")
        print("Goal pace over 82 games: {:.2f}".format(goals_18_19 / games_played_18_19 * 82))
        print("Predicted goal pace over 82 games: {:.2f}".format(pred_goals_18_19 / games_played_18_19 * 82))
        print("Games played:", games_played_18_19)
        print("\n")
        
    else:
        
        print("Player", name, "did not play in the 2018-2019 season.\n\n")

print("\n\n")
print("Overperformers in 2017-2018")
print("------------------------------")

for i in range(n):
    
    print("------------------------------")
    print("\n")
    
    # Overperformers
    j = np.argsort(differences_17_18)[N-i-1]
    player_id = pd.DataFrame(y_test.iloc[j]).T.index[0]
    
    name = pd.DataFrame(df_all.loc[player_id]).T["name"].iloc[0]
    
    # Make sure that player actually played next year
    if player_id in df2_all.index.tolist():
        
        games_played_17_18 = df_all["games_played"].loc[player_id]
        games_played_18_19 = df2_all["games_played"].loc[player_id]
        
        goals_17_18 = pd.DataFrame(df_all.loc[player_id]).T["I_F_goals"].iloc[0]
        goals_18_19 = pd.DataFrame(df2_all.loc[player_id]).T["I_F_goals"].iloc[0]
        pred_goals_17_18 = pipe_17_18_Lasso.predict(pd.DataFrame(df_all.loc[player_id]).T)[0]
        pred_goals_18_19 = pipe_18_19_Lasso.predict(pd.DataFrame(df2_all.loc[player_id]).T)[0]


        print("Player:", name)
        print("Player ID:", player_id)
        print("\n")
        
        print("2017-2018 Season")
        print("----------------------")
        
        print("Goal pace over 82 games: {:.2f}".format(goals_17_18 / games_played_17_18 * 82))
        print("Predicted goal pace over 82 games: {:.2f}".format(pred_goals_17_18 / games_played_17_18 * 82))
        print("Games played:", games_played_17_18)
        print("\n")

        print("2018-2019 Season")
        print("----------------------")
        print("Goal pace over 82 games: {:.2f}".format(goals_18_19 / games_played_18_19 * 82))
        print("Predicted goal pace over 82 games: {:.2f}".format(pred_goals_18_19 / games_played_18_19 * 82))
        print("Games played:", games_played_18_19)
        print("\n")
        
    else:
        
        print("Player", name, "did not play in the 2018-2019 season.\n\n")

Underperformers in 2017-2018
------------------------------
------------------------------


Player: Danny O'Regan
Player ID: 8476982


2017-2018 Season
----------------------
Goal pace over 82 games: 0.00
Predicted goal pace over 82 games: 44.10
Games played: 21


2018-2019 Season
----------------------
Goal pace over 82 games: 0.00
Predicted goal pace over 82 games: -512.66
Games played: 1


------------------------------


Player: Xavier Ouellet
Player ID: 8476443


2017-2018 Season
----------------------
Goal pace over 82 games: 0.00
Predicted goal pace over 82 games: 20.16
Games played: 45


2018-2019 Season
----------------------
Goal pace over 82 games: 0.00
Predicted goal pace over 82 games: -18.43
Games played: 19


------------------------------


Player: Dominic Turgeon
Player ID: 8477994


2017-2018 Season
----------------------
Goal pace over 82 games: 0.00
Predicted goal pace over 82 games: 181.31
Games played: 5


2018-2019 Season
----------------------
Goal pace over 82